<a href="https://colab.research.google.com/github/Blockchain-Framework/bitcoin-anomaly-analysis/blob/develop/Notebooks/Bitcoin_create_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Inputs

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/bitcoin_address_info_inputs2.csv")

In [ ]:
df.head()

,account,hash,time_x,value,value_usd,spending_value_usd,time_y,is_coinbase,block_id,time
0,3HRZjedwF2AJejNTtgznWnas4E6froNP5r,1f4b0ee916d74b4658c555e22b1ba5a1b0a5bfe18a0db1...,2018-12-31 23:50:39,36845206,1411.2009,1398.9056,2019-01-01 00:03:10,0,556459,2019-01-01 00:03:10
1,3HRZjedwF2AJejNTtgznWnas4E6froNP5r,a11cc6ecd95c8ac79098af1875a987ae5417410e805240...,2018-12-31 23:50:39,271339279,10392.5117,10301.9658,2019-01-01 00:03:10,0,556459,2019-01-01 00:03:10
2,32RQQ1ZVwvhoLhre727pH81g2BdQc77njA,1e9d790c147104ec63064d076f84ff72c0c619f1ccb7b7...,2018-12-31 21:23:24,38350000,1468.8357,1456.0383,2019-01-01 00:03:10,0,556459,2019-01-01 00:03:10
3,175xKXTfcLiXgX7XqxLCaskBzW4Qs3nWAM,d50e0bf3a4ca28f14c6b9539e8da85804b1425efecee2c...,2018-12-31 23:50:39,276291300,10582.1777,10489.9795,2019-01-01 00:03:10,0,556459,2019-01-01 00:03:10
4,3HRZjedwF2AJejNTtgznWnas4E6froNP5r,fad5d3798f5f4788bac3fd3ee0df649ef8914adcfa0cbc...,2019-01-01 00:03:10,270983892,10288.4727,10288.4727,2019-01-01 00:36:32,0,556460,2019-01-01 00:36:32


In [ ]:
df.dtypes

account                object
hash                   object
time_x                 object
value                   int64
value_usd             float64
spending_value_usd    float64
time_y                 object
is_coinbase             int64
block_id                int64
time                   object
dtype: object

In [ ]:
df['account'] = df['account'].astype('category')
df['hash'] = df['hash'].astype('category')
df['value'] = df['value'].astype('int64')
df['value_usd'] = df['value_usd'].astype('float64')
df['spending_value_usd'] = df['spending_value_usd'].astype('float64')
df['block_id'] = df['block_id'].astype('int64')
df['time'] = pd.to_datetime(df['time'])

In [ ]:

def compute_statistics(group, column):
    return {
        'mean': group[column].mean(),
        'median': group[column].median(),
        'variance': group[column].var(),
        'maximum': group[column].max(),
        'minimum': group[column].min(),
        'skewness': group[column].skew(),
        'kurtosis': group[column].kurt(),
        '25th_percentile': group[column].quantile(0.25),
        '75th_percentile': group[column].quantile(0.75),
        'range': group[column].max() - group[column].min()
    }

# Preparing for the final DataFrame
results = []

# Computing statistics for each account
for name, group in df.groupby('account'):
    group = group.sort_values(by='time')
    group['time_diff'] = group['time'].diff().dt.total_seconds().fillna(0)

    stat_dict = {'account': name}
    for column in ['value', 'value_usd', 'spending_value_usd', 'time_diff']:
        stats = compute_statistics(group, column)
        for key, value in stats.items():
            stat_dict[f'{column}_{key}'] = value
    stat_dict.update({
            'transaction_count': len(group),
        })

    results.append(stat_dict)


In [ ]:
# Creating the final DataFrame
final_df = pd.DataFrame(results)


In [ ]:
final_df.head()

,account,value_mean,value_median,value_variance,value_maximum,value_minimum,value_skewness,value_kurtosis,value_25th_percentile,value_75th_percentile,...,time_diff_median,time_diff_variance,time_diff_maximum,time_diff_minimum,time_diff_skewness,time_diff_kurtosis,time_diff_25th_percentile,time_diff_75th_percentile,time_diff_range,transaction_count
0,112aMbzaifSzz17dt4iieNBNRaNMPMsBhN,4.126654e+06,3775772.0,1.430222e+13,10000000,100000,0.960194,1.074525,1757500.0,5000000.0,...,899043.0,1.078162e+14,24258799.0,0.0,1.964865,3.866357,789.0,5781539.0,24258799.0,5
1,112uqsxthsgX3bBSJzTaZngaNQhPfHnoa6,6.914500e+06,6914500.0,6.119498e+13,12446000,1383000,NaN,NaN,4148750.0,9680250.0,...,0.0,0.000000e+00,0.0,0.0,NaN,NaN,0.0,0.0,0.0,2
2,1131uSp9Aq2vf5kfxUCaP68nFsBoSoKByk,3.880419e+06,1104075.0,6.499291e+13,53137394,547,3.927683,17.282748,483815.5,3048058.0,...,0.0,1.100215e+10,690650.0,0.0,4.107071,19.098731,0.0,2847.0,690650.0,111
3,1136NPxiQgXe7a4hXoUmD8XwuUyBrwBjni,3.068000e+06,3068000.0,NaN,3068000,3068000,NaN,NaN,3068000.0,3068000.0,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1
4,113EN2Wep2eXUY7M7H4XNGGE9evGnVowR4,3.719556e+07,37195558.0,NaN,37195558,37195558,NaN,NaN,37195558.0,37195558.0,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1


In [ ]:
final_df.shape

(44730, 42)

In [ ]:
final_df['account'].value_counts()

112aMbzaifSzz17dt4iieNBNRaNMPMsBhN            1
3CvS65zs3xdhXdMFm3kGWUV8fAbuY66xK4            1
3Cv5N5qNsLuuAb3Ri5pNTJorsCqMi61Y9Z            1
3Cv5reGPRwzKHLGhRg8xqHoNKnZA4BW3dn            1
3Cv93NMVg8uC9yDA9dMi4RMf2ECufLRe2P            1
                                             ..
1Nqn1wjLqEVQg2X96XGoAySFLwQ2Rj8MWc            1
1Nqt1CBWCfq126EYrs3nmzsiTysUbu7hEK            1
1NqyHq7wD4iYuApEr29zpCuhDkCjkw7Eff            1
1Nr1SWCbHpKjmfjSskwjbsY4Y1BntCeAcr            1
bc1qzspr46dq4p6l56mj4g8zk6s0mtkmk4v8fr3gm4    1
Name: account, Length: 44730, dtype: int64

In [ ]:
final_df.isnull().sum()

account                                   0
value_mean                                0
value_median                              0
value_variance                        28779
value_maximum                             0
value_minimum                             0
value_skewness                        34183
value_kurtosis                        36688
value_25th_percentile                     0
value_75th_percentile                     0
value_range                               0
value_usd_mean                            0
value_usd_median                          0
value_usd_variance                    28779
value_usd_maximum                         0
value_usd_minimum                         0
value_usd_skewness                    34183
value_usd_kurtosis                    36688
value_usd_25th_percentile                 0
value_usd_75th_percentile                 0
value_usd_range                           0
spending_value_usd_mean                   0
spending_value_usd_median       

In [ ]:
df['account'].value_counts()

3HRZjedwF2AJejNTtgznWnas4E6froNP5r            356568
32Mf4hkRXAxhrkY2EjrzVcX8diTPKVEfhA            261326
3E5NsFYg9H64TSB9MroJCCZJG7dJ179Yge            212884
3ABE4BZkwv2ubYYGhUtNxL57gMwenUEuNW            104715
3Q3fELB57fJAd9kUJcUkMZfxm7ez55fN8p             76423
                                               ...  
35eGHHZu7t9NjcdZR5Se1bTd6NW1J7wjT5                 1
35e4hsUQwJtRXhzBdFi9nWYN398xVTATZT                 1
35dx5gSYpAJq9ScGj916ofEuwn81q9Hzzj                 1
35duTEpvi6Sh3UxaAtg31ybwx3zRF8NUwH                 1
bc1qzspr46dq4p6l56mj4g8zk6s0mtkmk4v8fr3gm4         1
Name: account, Length: 44730, dtype: int64

In [ ]:
record_counts = df['account'].value_counts()

threshold = 4
accounts_with_few_records = record_counts[record_counts < threshold].index.tolist()
len(accounts_with_few_records)

36688

In [ ]:
final_df.to_csv('/content/drive/MyDrive/bitcoin_input_processed.csv', index=False)

# Outputs

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/bitcoin_address_info_outputs2.csv")

In [ ]:
df1.dtypes

account         object
hash            object
time_x          object
value            int64
value_usd      float64
time_y          object
is_coinbase      int64
block_id         int64
time            object
dtype: object

In [ ]:
df1['account'] = df1['account'].astype('category')
df1['hash'] = df1['hash'].astype('category')
df1['value'] = df1['value'].astype('int64')
df1['value_usd'] = df1['value_usd'].astype('float64')
df1['block_id'] = df1['block_id'].astype('int64')
df1['time'] = pd.to_datetime(df1['time'])

In [ ]:

def compute_statistics(group, column):
    return {
        'mean': group[column].mean(),
        'median': group[column].median(),
        'variance': group[column].var(),
        'maximum': group[column].max(),
        'minimum': group[column].min(),
        'skewness': group[column].skew(),
        'kurtosis': group[column].kurt(),
        '25th_percentile': group[column].quantile(0.25),
        '75th_percentile': group[column].quantile(0.75),
        'range': group[column].max() - group[column].min()
    }

# Preparing for the final DataFrame
results1 = []

# Computing statistics for each account
for name, group in df1.groupby('account'):
    group = group.sort_values(by='time')
    group['time_diff'] = group['time'].diff().dt.total_seconds().fillna(0)

    stat_dict = {'account': name}
    for column in ['value', 'value_usd', 'time_diff']:
        stats = compute_statistics(group, column)
        for key, value in stats.items():
            stat_dict[f'{column}_{key}'] = value
    stat_dict.update({
            'transaction_count': len(group),
        })

    results1.append(stat_dict)


In [ ]:
final_df1 = pd.DataFrame(results1)

In [ ]:
final_df1.head()

,account,value_mean,value_median,value_variance,value_maximum,value_minimum,value_skewness,value_kurtosis,value_25th_percentile,value_75th_percentile,...,time_diff_median,time_diff_variance,time_diff_maximum,time_diff_minimum,time_diff_skewness,time_diff_kurtosis,time_diff_25th_percentile,time_diff_75th_percentile,time_diff_range,transaction_count
0,1122NYbAT2KkZDZ5TFvGy4D2Ut7eYfx4en,1.000000e+04,10000.0,NaN,10000,10000,NaN,NaN,10000.0,10000.0,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1
1,112aMbzaifSzz17dt4iieNBNRaNMPMsBhN,4.126654e+06,3775772.0,1.430222e+13,10000000,100000,0.960194,1.074525,1757500.0,5000000.0,...,51199.0,3.195106e+14,40034756.0,0.0,2.235975,4.999665,23241.0,188317.0,40034756.0,5
2,112uqsxthsgX3bBSJzTaZngaNQhPfHnoa6,6.914500e+06,6914500.0,6.119498e+13,12446000,1383000,NaN,NaN,4148750.0,9680250.0,...,532.0,5.660480e+05,1064.0,0.0,NaN,NaN,266.0,798.0,1064.0,2
3,1131uSp9Aq2vf5kfxUCaP68nFsBoSoKByk,4.339443e+06,1167722.0,7.563074e+13,53137394,547,3.392790,12.405094,498441.0,3285207.0,...,11521.0,3.419762e+09,451766.0,0.0,4.170106,24.283287,2066.0,42373.0,451766.0,113
4,1136NPxiQgXe7a4hXoUmD8XwuUyBrwBjni,3.068000e+06,3068000.0,NaN,3068000,3068000,NaN,NaN,3068000.0,3068000.0,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1


In [ ]:
final_df1.shape

(44875, 32)

In [ ]:
final_df1.isnull().sum()

account                          0
value_mean                       0
value_median                     0
value_variance               26861
value_maximum                    0
value_minimum                    0
value_skewness               31765
value_kurtosis               35679
value_25th_percentile            0
value_75th_percentile            0
value_range                      0
value_usd_mean                   0
value_usd_median                 0
value_usd_variance           26861
value_usd_maximum                0
value_usd_minimum                0
value_usd_skewness           31765
value_usd_kurtosis           35679
value_usd_25th_percentile        0
value_usd_75th_percentile        0
value_usd_range                  0
time_diff_mean                   0
time_diff_median                 0
time_diff_variance           26861
time_diff_maximum                0
time_diff_minimum                0
time_diff_skewness           31765
time_diff_kurtosis           35679
time_diff_25th_perce

In [ ]:
record_counts = df1['account'].value_counts()

threshold = 3
accounts_with_few_records = record_counts[record_counts < threshold].index.tolist()
len(accounts_with_few_records)

31765

In [ ]:
record_counts = df1['account'].value_counts()

threshold = 4
accounts_with_few_records = record_counts[record_counts < threshold].index.tolist()
len(accounts_with_few_records)

35679

In [ ]:
record_counts = df1['account'].value_counts()

threshold = 2
accounts_with_few_records = record_counts[record_counts < threshold].index.tolist()
len(accounts_with_few_records)

26861

In [ ]:
final_df1.to_csv('/content/drive/MyDrive/bitcoin_output_processed.csv', index=False)

# Combining Inputs & Outputs

In [ ]:
final_df.columns

Index(['account', 'value_mean', 'value_median', 'value_variance',
       'value_maximum', 'value_minimum', 'value_skewness', 'value_kurtosis',
       'value_25th_percentile', 'value_75th_percentile', 'value_range',
       'value_usd_mean', 'value_usd_median', 'value_usd_variance',
       'value_usd_maximum', 'value_usd_minimum', 'value_usd_skewness',
       'value_usd_kurtosis', 'value_usd_25th_percentile',
       'value_usd_75th_percentile', 'value_usd_range',
       'spending_value_usd_mean', 'spending_value_usd_median',
       'spending_value_usd_variance', 'spending_value_usd_maximum',
       'spending_value_usd_minimum', 'spending_value_usd_skewness',
       'spending_value_usd_kurtosis', 'spending_value_usd_25th_percentile',
       'spending_value_usd_75th_percentile', 'spending_value_usd_range',
       'time_diff_mean', 'time_diff_median', 'time_diff_variance',
       'time_diff_maximum', 'time_diff_minimum', 'time_diff_skewness',
       'time_diff_kurtosis', 'time_diff_25th_per

In [ ]:
final_df1.columns

Index(['account', 'value_mean', 'value_median', 'value_variance',
       'value_maximum', 'value_minimum', 'value_skewness', 'value_kurtosis',
       'value_25th_percentile', 'value_75th_percentile', 'value_range',
       'value_usd_mean', 'value_usd_median', 'value_usd_variance',
       'value_usd_maximum', 'value_usd_minimum', 'value_usd_skewness',
       'value_usd_kurtosis', 'value_usd_25th_percentile',
       'value_usd_75th_percentile', 'value_usd_range', 'time_diff_mean',
       'time_diff_median', 'time_diff_variance', 'time_diff_maximum',
       'time_diff_minimum', 'time_diff_skewness', 'time_diff_kurtosis',
       'time_diff_25th_percentile', 'time_diff_75th_percentile',
       'time_diff_range', 'transaction_count'],
      dtype='object')

In [ ]:
final_df = final_df.add_prefix('input_')
final_df1 = final_df1.add_prefix('output_')

# Rename 'key' columns back to original, as we want to merge on this column
final_df = final_df.rename(columns={'input_account': 'account'})
final_df1 = final_df1.rename(columns={'output_account': 'account'})

In [ ]:
final_result = pd.merge(final_df, final_df1, on='account', how='outer')

In [ ]:
final_result.head()

,account,input_value_mean,input_value_median,input_value_variance,input_value_maximum,input_value_minimum,input_value_skewness,input_value_kurtosis,input_value_25th_percentile,input_value_75th_percentile,...,output_time_diff_median,output_time_diff_variance,output_time_diff_maximum,output_time_diff_minimum,output_time_diff_skewness,output_time_diff_kurtosis,output_time_diff_25th_percentile,output_time_diff_75th_percentile,output_time_diff_range,output_transaction_count
0,112aMbzaifSzz17dt4iieNBNRaNMPMsBhN,4.126654e+06,3775772.0,1.430222e+13,10000000.0,100000.0,0.960194,1.074525,1757500.0,5000000.0,...,51199.0,3.195106e+14,40034756.0,0.0,2.235975,4.999665,23241.0,188317.0,40034756.0,5.0
1,112uqsxthsgX3bBSJzTaZngaNQhPfHnoa6,6.914500e+06,6914500.0,6.119498e+13,12446000.0,1383000.0,NaN,NaN,4148750.0,9680250.0,...,532.0,5.660480e+05,1064.0,0.0,NaN,NaN,266.0,798.0,1064.0,2.0
2,1131uSp9Aq2vf5kfxUCaP68nFsBoSoKByk,3.880419e+06,1104075.0,6.499291e+13,53137394.0,547.0,3.927683,17.282748,483815.5,3048058.0,...,11521.0,3.419762e+09,451766.0,0.0,4.170106,24.283287,2066.0,42373.0,451766.0,113.0
3,1136NPxiQgXe7a4hXoUmD8XwuUyBrwBjni,3.068000e+06,3068000.0,NaN,3068000.0,3068000.0,NaN,NaN,3068000.0,3068000.0,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1.0
4,113EN2Wep2eXUY7M7H4XNGGE9evGnVowR4,3.719556e+07,37195558.0,NaN,37195558.0,37195558.0,NaN,NaN,37195558.0,37195558.0,...,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1.0


In [ ]:
final_result.columns

Index(['account', 'input_value_mean', 'input_value_median',
       'input_value_variance', 'input_value_maximum', 'input_value_minimum',
       'input_value_skewness', 'input_value_kurtosis',
       'input_value_25th_percentile', 'input_value_75th_percentile',
       'input_value_range', 'input_value_usd_mean', 'input_value_usd_median',
       'input_value_usd_variance', 'input_value_usd_maximum',
       'input_value_usd_minimum', 'input_value_usd_skewness',
       'input_value_usd_kurtosis', 'input_value_usd_25th_percentile',
       'input_value_usd_75th_percentile', 'input_value_usd_range',
       'input_spending_value_usd_mean', 'input_spending_value_usd_median',
       'input_spending_value_usd_variance', 'input_spending_value_usd_maximum',
       'input_spending_value_usd_minimum', 'input_spending_value_usd_skewness',
       'input_spending_value_usd_kurtosis',
       'input_spending_value_usd_25th_percentile',
       'input_spending_value_usd_75th_percentile',
       'input_spend

In [ ]:
final_result.isnull().sum()

account                                 0
input_value_mean                      280
input_value_median                    280
input_value_variance                29059
input_value_maximum                   280
                                    ...  
output_time_diff_kurtosis           35814
output_time_diff_25th_percentile      135
output_time_diff_75th_percentile      135
output_time_diff_range                135
output_transaction_count              135
Length: 73, dtype: int64

In [ ]:
final_result[['input_transaction_count', 'output_transaction_count']].isnull().sum()

input_transaction_count     280
output_transaction_count    135
dtype: int64

In [ ]:
null_counts = final_result[
    final_result['input_transaction_count'].isnull() &
    final_result['output_transaction_count'].isnull()
].shape[0]

print(null_counts)

0


In [ ]:
final_result.to_csv('/content/drive/MyDrive/bitcoin_final.csv', index=False)

In [ ]:
babd_sample = pd.read_csv("/content/drive/MyDrive/combined_babd_sample.csv")

In [ ]:
final_result_label = pd.merge(final_result, babd_sample, on='account', how='left')

In [ ]:
final_result.shape

(45010, 73)

In [ ]:
final_result_label.shape

(46612, 74)

In [ ]:
babd_group = babd_sample.groupby('account')


In [ ]:
group_count_with_diff_labels = sum(group['label'].nunique() > 1 for _, group in babd_group)

group_count_with_diff_labels

1065

In [ ]:

non_unique_label_accounts = babd_group.filter(lambda x: x['label'].nunique() > 1)['account'].unique()

non_unique_label_accounts

array(['19krxcYxs9dyuyCxkFdmvB4bXfKC46PB5x',
       '113ioG4RFd9pRVfJVKauCbg7QH3oUe6URa',
       '114QyEQGVvamQTVDHBXB9DqDnNBSyVvvMD', ...,
       '38vfecvCWN1o7yJrKjUXpX9iKWQuj992Rv',
       '3JyL33QhbtxZMvw3T5qHXPPQtMxRbwWJk3',
       'bc1qyep0rradh8z7ha3fhzauy3x2fh6teglkd7zkx7'], dtype=object)

In [ ]:
def select_label(group):
    label_counts = group['label'].value_counts()
    # If there's a most frequent label, use that, otherwise use the first one
    return label_counts.idxmax() if label_counts.max() > 1 else group['label'].iloc[0]

# Grouping by 'account' and applying the label selection function
selected_labels = babd_sample.groupby('account').apply(select_label).reset_index(name='selected_label')

# Merging the selected labels back with the original DataFrame and dropping duplicates
filtered_babd_sample = pd.merge(babd_sample.drop('label', axis=1), selected_labels, on='account').drop_duplicates('account')


In [ ]:
babd_sample.shape

(46612, 2)

In [ ]:
filtered_babd_sample.shape

(45010, 2)

In [ ]:
filtered_babd_sample.head()

,account,selected_label
0,1MYVum5nsg9irQMrUsh4sxE8obfbSwTmBL,-1
1,19DrAW7qQrashPBg7r6tWkTxD7sL1cA82B,-1
2,1gpfEvpEZ2qfz8vbRoRtAQoqnpmYTFNWU,-1
3,336eBgmKhxYNKCG44xXnAkhH5PVhpo7JsF,-1
4,1N8k4iXndXpWNdYUKkF3dLybxQZ6pGwT69,-1


In [ ]:
filtered_babd_sample['selected_label'].value_counts()

 2     13861
-1     11653
 11    11348
 0      8148
Name: selected_label, dtype: int64

In [ ]:
filtered_babd_sample_gorup = filtered_babd_sample.groupby('account')

In [ ]:
group_count_with_diff_labels = sum(group['selected_label'].nunique() > 1 for _, group in filtered_babd_sample_gorup)

group_count_with_diff_labels

0

In [ ]:
filtered_babd_sample = filtered_babd_sample.rename(columns={"selected_label": "label"})

In [ ]:
final_df = pd.read_csv("/content/drive/MyDrive/bitcoin_final.csv")

In [ ]:
final_df.columns

Index(['account', 'input_value_mean', 'input_value_median',
       'input_value_variance', 'input_value_maximum', 'input_value_minimum',
       'input_value_skewness', 'input_value_kurtosis',
       'input_value_25th_percentile', 'input_value_75th_percentile',
       'input_value_range', 'input_value_usd_mean', 'input_value_usd_median',
       'input_value_usd_variance', 'input_value_usd_maximum',
       'input_value_usd_minimum', 'input_value_usd_skewness',
       'input_value_usd_kurtosis', 'input_value_usd_25th_percentile',
       'input_value_usd_75th_percentile', 'input_value_usd_range',
       'input_spending_value_usd_mean', 'input_spending_value_usd_median',
       'input_spending_value_usd_variance', 'input_spending_value_usd_maximum',
       'input_spending_value_usd_minimum', 'input_spending_value_usd_skewness',
       'input_spending_value_usd_kurtosis',
       'input_spending_value_usd_25th_percentile',
       'input_spending_value_usd_75th_percentile',
       'input_spend

In [ ]:
final_df.shape

(45010, 73)

In [ ]:
final_result_label = pd.merge(final_df, filtered_babd_sample, on='account', how='left')

In [ ]:
final_result_label.shape

(45010, 74)

In [ ]:
final_result_label.head()

,account,input_value_mean,input_value_median,input_value_variance,input_value_maximum,input_value_minimum,input_value_skewness,input_value_kurtosis,input_value_25th_percentile,input_value_75th_percentile,...,output_time_diff_variance,output_time_diff_maximum,output_time_diff_minimum,output_time_diff_skewness,output_time_diff_kurtosis,output_time_diff_25th_percentile,output_time_diff_75th_percentile,output_time_diff_range,output_transaction_count,label
0,112aMbzaifSzz17dt4iieNBNRaNMPMsBhN,4.126654e+06,3775772.0,1.430222e+13,10000000.0,100000.0,0.960194,1.074525,1757500.0,5000000.0,...,3.195106e+14,40034756.0,0.0,2.235975,4.999665,23241.0,188317.0,40034756.0,5.0,-1
1,112uqsxthsgX3bBSJzTaZngaNQhPfHnoa6,6.914500e+06,6914500.0,6.119498e+13,12446000.0,1383000.0,NaN,NaN,4148750.0,9680250.0,...,5.660480e+05,1064.0,0.0,NaN,NaN,266.0,798.0,1064.0,2.0,0
2,1131uSp9Aq2vf5kfxUCaP68nFsBoSoKByk,3.880419e+06,1104075.0,6.499291e+13,53137394.0,547.0,3.927683,17.282748,483815.5,3048058.0,...,3.419762e+09,451766.0,0.0,4.170106,24.283287,2066.0,42373.0,451766.0,113.0,0
3,1136NPxiQgXe7a4hXoUmD8XwuUyBrwBjni,3.068000e+06,3068000.0,NaN,3068000.0,3068000.0,NaN,NaN,3068000.0,3068000.0,...,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1.0,0
4,113EN2Wep2eXUY7M7H4XNGGE9evGnVowR4,3.719556e+07,37195558.0,NaN,37195558.0,37195558.0,NaN,NaN,37195558.0,37195558.0,...,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1.0,0


In [ ]:
final_result_label['label'].value_counts()

 2     13861
-1     11653
 11    11348
 0      8148
Name: label, dtype: int64

In [ ]:
# -1 - good
# 0 - blackcmail
# 2 - darknet
# 11 - ponzi shceme

In [ ]:
label_mapping = {-1: 0, 0: 1, 2: 2, 11: 3}
final_result_label['label'] = final_result_label['label'].map(label_mapping)

In [ ]:
final_result_label['label'].value_counts()

2    13861
0    11653
3    11348
1     8148
Name: label, dtype: int64

In [ ]:
final_result_label.to_csv('/content/drive/MyDrive/bitcoin_final_label_dataset.csv', index=False)